### IMPORTS

In [1]:
import numpy as np
import scipy as sp

$\textbf{QUESTION 1}$
$$
\textbf{Linear Program Objective Function:}\\
\text{min}\ 3x_1 + 2x_2
\\~\\
\textbf{Constraints:}\\
2x_1 + x_2 \ge 4\\
x_1 + x_2 \ge 3\\
x_1 + 2x_2 \le 10\\
x_1 \ge 0\\
x_2 \ge 0
\\~\\
\text{The feasible region is the shaded portion in the below graph.}
$$
<p align="center">
<img src="/home/leonardo_leads/Documents/SchoolDocs/ga_tech_masters/omscs_ml/fall2023/deterministic_optimization/lectures/week8Modules15and16/homework/images/Q1.png" width="600" height="600" />
</p>

$\textbf{QUESTION 2}$

$$
\textbf{Standard Form Linear Program:}\\
\textbf{Objective Function:}\\
\text{min}\ 3x_1 + 2x_2
\\~\\
\textbf{Constraints:}\\
2x_1 + x_2 - x_3 = 4\\
x_1 + x_2 - x_4 = 3\\
x_1 + 2x_2 +x_5 = 10\\
x_1, x_2, x_3, x_4, x_5 \ge 0
\\~\\
x = 
\begin{bmatrix}
x_1 & x_2 & x_3 & x_4 & x_5
\end{bmatrix}^T
\\~\\
c = 
\begin{bmatrix}
3 & 2 & 0 & 0 & 0
\end{bmatrix}^T
\\~\\
A =
\begin{bmatrix}
2 & 1 & -1 & 0 & 0\\
1 & 1 & 0 & -1 & 0\\
1 & 2 & 0 & 0 & 1
\end{bmatrix}
\\~\\
b = 
\begin{bmatrix}
4 & 3 & 10
\end{bmatrix}
$$

$\textbf{QUESTION 3}$
$$
\textbf{PART A}\\
$$
$
\text{ITERATION 1}
$

In [2]:
c = np.array([3, 2, 0, 0, 0]).T
A = np.matrix([[2, 1, -1, 0, 0],
               [1, 1, 0, -1, 0],
               [1, 2, 0, 0, 1],])
b = np.array([4, 3, 10]).T

# basis matrix is the BFS
basis = np.array([A[:, [0, 2, 3]]])
b_inv = np.linalg.inv(basis)
x_b = b_inv @ b
x_n = np.array([0, 0]).T
c_b = c[[0, 2, 3]]
c_n = c[[1, 4]]
Aij = [A[:, 1], A[:, 4]]
k = 1
print(f"Current Iteration: {k=}\nBasis (B):\n{basis=}\n{b_inv=}")

if np.all(x_b >= 0):
    print(f"Current basic solution is a BFS:\n{x_b=}\n{x_n=}")

    # Generalize c_n[] and A1/A2
    c_n_bari = np.squeeze(np.asarray(c_n[0] - c_b.T @ b_inv @ Aij[0]))
    c_n_barj = np.squeeze(np.asarray(c_n[1] - c_b.T @ b_inv @ Aij[1]))

    # Check if BFS is optimal solution
    if c_n_bari < 0 or c_n_barj < 0:            
        print(f"Current BFS is not optimal:\n{c_n_bari=}\n{c_n_barj=}\nx5 enters.\nx4 exits.")

        # Choose A5 to enter the basis
        d_b = np.squeeze(np.asarray(-b_inv @ Aij[1]))
        d_n = np.array([0, 1])
        
        # Check if optimal solutions are unbounded
        if np.any(d_b < 0):
            print(f"Do not have an unbounded optimal solution:\n{d_b=}\n{d_n=}\n")

            # Minimum-Ratio Test
            theta = np.min(x_b / -d_b)
            theta_b, theta_n = np.squeeze(np.asarray(x_b / -d_b)), np.array([0, theta])
            print(f"Theta Values:\n{theta=}\n{theta_b=}\n{theta_n=}")

            # # Bland's Rule
            # '''
            # Choose the entering nonbasic variable x_j s.t. j is the smallest index with c_j^bar < 0
            # Choose the exiting basic variable x_i s.t. i is the smallest index that achieves the min ratio
            # '''
            # c_n_bar = [c_n_bar1, c_n_bar2]
            # entering_var = min(c_n_bar)
            # entering_var_index = c_n_bar.index(entering_var)
            # exiting_var = min(theta_b)
            # exiting_var_index = np.where(theta_b == exiting_var)[0][0]
            # print(f"{entering_var_index=}\n{exiting_var_index=}")
            
            # New data: NEEDS TO BE GENERALIZED
            # basis[0][:, exiting_var_index] = np.squeeze(np.asarray(A[:, entering_var_index]))
            # print(f"{'&'*10}\n{basis=}\n{'&'*10}")
            # b_inv = np.linalg.inv(basis)
            # c_b[exiting_var_index] = c[entering_var_index]
            # c_n[entering_var_index] = c[exiting_var_index + 2]
            # Aij[entering_var_index] = A[:, exiting_var_index + 2]
            # x_b = b_inv @ b
            # print(f"{x_b=}")

        else:
            print("Do have unbounded optimal solution.")
            # break
        
    else: print(f"Current BFS is optimal:\n{c_n_bari=}\n{c_n_barj=}")
        # break

else:
    print("Current basic solution is not a BFS")
    # break

k += 1


Current Iteration: k=1
Basis (B):
basis=array([[[ 2, -1,  0],
        [ 1,  0, -1],
        [ 1,  0,  0]]])
b_inv=array([[[ 0.,  0.,  1.],
        [-1.,  0.,  2.],
        [ 0., -1.,  1.]]])
Current basic solution is a BFS:
x_b=array([[10., 16.,  7.]])
x_n=array([0, 0])
Current BFS is not optimal:
c_n_bari=array(-4.)
c_n_barj=array(-3.)
x5 enters.
x4 exits.
Do not have an unbounded optimal solution:
d_b=array([-1., -2., -1.])
d_n=array([0, 1])

Theta Values:
theta=7.0
theta_b=array([10.,  8.,  7.])
theta_n=array([0., 7.])


$
\text{ITERATION 2}
$

In [3]:
# basis matrix is the BFS
basis = np.array([A[:, [0, 2, 4]]])
b_inv = np.linalg.inv(basis)
x_b = b_inv @ b
c_b = c[[0, 2, 4]]
c_n = c[[1, 3]]
Aij = [A[:, 1], A[:, 3]]
print(f"Current Iteration: {k=}\nBasis (B):\n{basis=}\n{b_inv=}")

if np.all(x_b >= 0):
    print(f"Current basic solution is a BFS:\n{x_b=}\n{x_n=}")

    # Generalize c_n[] and A1/A2
    c_n_bari = np.squeeze(np.asarray(c_n[0] - c_b.T @ b_inv @ Aij[0]))
    c_n_barj = np.squeeze(np.asarray(c_n[1] - c_b.T @ b_inv @ Aij[1]))

    # Check if BFS is optimal solution
    if c_n_bari < 0 or c_n_barj < 0:            
        print(f"Current BFS is not optimal:\n{c_n_bari=}\n{c_n_barj=}\nx2 enters.\nx3 exits.")

        # Choose A5 to enter the basis
        d_b = np.squeeze(np.asarray(-b_inv @ Aij[0]))
        d_n = np.array([1, 0])
        
        # Check if optimal solutions are unbounded
        if np.any(d_b < 0):
            print(f"Do not have an unbounded optimal solution:\n{d_b=}\n{d_n=}\n")

            # Minimum-Ratio Test
            theta = np.min(x_b / -d_b)
            theta_b, theta_n = np.squeeze(np.asarray(x_b / -d_b)), np.array([0, theta])
            print(f"Theta Values:\n{theta=}\n{theta_b=}\n{theta_n=}")

        else: print("Do have unbounded optimal solution.")
        
    else: print(f"Current BFS is optimal:\n{c_n_bari=}\n{c_n_barj=}")

else: print("Current basic solution is not a BFS")

k += 1

Current Iteration: k=2
Basis (B):
basis=array([[[ 2, -1,  0],
        [ 1,  0,  0],
        [ 1,  0,  1]]])
b_inv=array([[[ 0.,  1.,  0.],
        [-1.,  2.,  0.],
        [ 0., -1.,  1.]]])
Current basic solution is a BFS:
x_b=array([[3., 2., 7.]])
x_n=array([0, 0])
Current BFS is not optimal:
c_n_bari=array(-1.)
c_n_barj=array(3.)
x2 enters.
x3 exits.
Do not have an unbounded optimal solution:
d_b=array([-1., -1., -1.])
d_n=array([1, 0])

Theta Values:
theta=2.0
theta_b=array([3., 2., 7.])
theta_n=array([0., 2.])


$
\text{ITERATION 3}
$

In [4]:
# basis matrix is the BFS
basis = np.array([A[:, [0, 1, 4]]])
b_inv = np.linalg.inv(basis)
x_b = b_inv @ b
c_b = c[[0, 1, 4]]
c_n = c[[2, 3]]
Aij = [A[:, 2], A[:, 3]]
print(f"Current Iteration: {k=}\nBasis (B):\n{basis=}\n{b_inv=}")

if np.all(x_b >= 0):
    print(f"Current basic solution is a BFS:\n{x_b=}\n{x_n=}")

    # Generalize c_n[] and A1/A2
    c_n_bari = np.squeeze(np.asarray(c_n[0] - c_b.T @ b_inv @ Aij[0]))
    c_n_barj = np.squeeze(np.asarray(c_n[1] - c_b.T @ b_inv @ Aij[1]))

    # Check if BFS is optimal solution
    if c_n_bari < 0 or c_n_barj < 0:            
        print(f"Current BFS is not optimal:\n{c_n_bari=}\n{c_n_barj=}\n")

        # Choose A5 to enter the basis
        d_b = np.squeeze(np.asarray(-b_inv @ Aij[0]))
        d_n = np.array([1, 0])
        
        # Check if optimal solutions are unbounded
        if np.any(d_b < 0):
            print(f"Do not have an unbounded optimal solution:\n{d_b=}\n{d_n=}\n")

            # Minimum-Ratio Test
            theta = np.min(x_b / -d_b)
            theta_b, theta_n = np.squeeze(np.asarray(x_b / -d_b)), np.array([0, theta])
            print(f"Theta Values:\n{theta=}\n{theta_b=}\n{theta_n=}")

        else: print("Do have unbounded optimal solution.")
        
    else: print(f"Current BFS is optimal:\n{x_b=}\n{x_n=}\n{c_n_bari=}\n{c_n_barj=}\nNot have an unbounded optimal solution.")

else: print("Current basic solution is not a BFS")

k += 1

Current Iteration: k=3
Basis (B):
basis=array([[[2, 1, 0],
        [1, 1, 0],
        [1, 2, 1]]])
b_inv=array([[[ 1., -1.,  0.],
        [-1.,  2.,  0.],
        [ 1., -3.,  1.]]])
Current basic solution is a BFS:
x_b=array([[1., 2., 5.]])
x_n=array([0, 0])
Current BFS is optimal:
x_b=array([[1., 2., 5.]])
x_n=array([0, 0])
c_n_bari=array(1.)
c_n_barj=array(1.)
Not have an unbounded optimal solution.


$
\textbf{Appendix}
$
$$
\textbf{Check With Scipy.Optimize.Linprog}\\
\text{I get the same results through SciPy that I get with manual Simplex}
$$

In [5]:
print(sp.optimize.linprog(A_eq=A, b_eq=b, c=c, method='simplex', options={'bland': True}))

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 7.0
       x: [ 1.000e+00  2.000e+00  0.000e+00  0.000e+00  5.000e+00]
     nit: 6


/tmp/ipykernel_785669/116169075.py:1: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  print(sp.optimize.linprog(A_eq=A, b_eq=b, c=c, method='simplex', options={'bland': True}))
